In [2]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [6]:
train_data = torchvision.datasets.CIFAR10( root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10( root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle=False, num_workers=2)

100.0%


In [7]:
image, label = train_data[0]

In [8]:
image.size()

torch.Size([3, 32, 32])

In [9]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [10]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 12, 5) # (12, 28, 28)
        self.pool = nn.MaxPool2d(2, 2) # (12, 14, 14)
        self.conv2 = nn.Conv2d(12, 24, 5) # (24, 10, 10) -> (24, 5, 5) -> Flatten (24 * 5 * 5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [11]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [12]:
for epoch in range(30):  # loop over the dataset multiple times
    print(f"Training epoch {epoch}....")

    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data
        
        optimizer.zero_grad()  # zero the parameter gradients

        outputs = net(inputs)  # forward pass
        
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()  # backward pass

        running_loss += loss.item()

    print(f"Loss: {running_loss / len(train_loader):.4f}")

Training epoch 0....
Loss: 1.6417
Training epoch 1....
Loss: 1.2275
Training epoch 2....
Loss: 1.0679
Training epoch 3....
Loss: 0.9666
Training epoch 4....
Loss: 0.8925
Training epoch 5....
Loss: 0.8311
Training epoch 6....
Loss: 0.7787
Training epoch 7....
Loss: 0.7348
Training epoch 8....
Loss: 0.6935
Training epoch 9....
Loss: 0.6589
Training epoch 10....
Loss: 0.6287
Training epoch 11....
Loss: 0.5971
Training epoch 12....
Loss: 0.5721
Training epoch 13....
Loss: 0.5506
Training epoch 14....
Loss: 0.5319
Training epoch 15....
Loss: 0.5252
Training epoch 16....
Loss: 0.5033
Training epoch 17....
Loss: 0.4894
Training epoch 18....
Loss: 0.4825
Training epoch 19....
Loss: 0.4703
Training epoch 20....
Loss: 0.4582
Training epoch 21....
Loss: 0.4536
Training epoch 22....
Loss: 0.4498
Training epoch 23....
Loss: 0.4466
Training epoch 24....
Loss: 0.4340
Training epoch 25....
Loss: 0.4423
Training epoch 26....
Loss: 0.4346
Training epoch 27....
Loss: 0.4210
Training epoch 28....
Loss: 0.

In [13]:
torch.save(net.state_dict(), 'trained_net.pth')

In [14]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<All keys matched successfully>

In [15]:
correct = 0
total = 0

net.eval()  # Set the model to evaluation mode

with torch.no_grad():  # Disable gradient calculation
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f}%')

Accuracy: 63.72%


In [17]:
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image

image_path = ['example_1.jpg', 'example_2.jpg']  # Replace with your image path
images = [load_image(path) for path in image_path]

net.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculation
    for image in images:
        outputs = net(image)
        _, predicted = torch.max(outputs.data, 1)

        print(f'Prediction: {class_names[predicted.item()]}')

Prediction: plane
Prediction: dog
